In [1]:
import sys
import os

current_directory = os.getcwd()
path = os.path.dirname(current_directory)
sys.path.append(path)
from Utils import *

%matplotlib widget
from ipywidgets import interact, interactive, widgets
from matplotlib.patches import Rectangle, Circle, Arrow

In [6]:
import glob
import os

import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable

import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, ArrayDataset, create_test_image_3d, DataLoader
from monai.transforms import (
    Transform,
    Compose,
    EnsureChannelFirst,
    EnsureChannelFirstd,
    LoadImage,
    Orientation,
    ConcatItemsd
)
from monaiUtils import MetatensorToSitk, SitkToMetatensor

# Load images
srcdir = os.getcwd()

print('Reading labeled images from: ' + srcdir)
images_1 = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_M.nii.gz')))
images_2 = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_P.nii.gz')))
labels = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_multi_label.nii.gz')))

# Create dictionary for MONAI
train_files = [
    {'image_1': image_name_1, 'image_2': image_name_2}
    for image_name_1, image_name_2 in zip(images_1, images_2)
]
img1_file_2d = train_files[0]['image_1']
img2_file_2d = train_files[0]['image_2']

img1_file_3d = train_files[1]['image_1']
img2_file_3d = train_files[1]['image_2']

# Load original image as SimpleITK object
sitk_input_2d = sitk.ReadImage(images_1[0], sitk.sitkFloat32)
show_image(sitk_input_2d, 'Loaded Mag 2D sitk image from nii')
sitk_input_3d = sitk.ReadImage(images_1[1], sitk.sitkFloat32)
show_image(sitk_input_3d, 'Loaded Mag 3D sitk image from nii')

print('SimpleITK object sizes')
print(sitk_input_2d.GetSize())
print(sitk_input_3d.GetSize())


# Get MONAI metatensor from sitk image
metatensor_2d = SitkToMetatensor(sitk_input_2d)
metatensor_3d = SitkToMetatensor(sitk_input_3d)
print('Metatensors for sitk before EnsureChannelFirst')
print(metatensor_2d.shape)
print(metatensor_3d.shape)

# Make it multi-channel
multiChannel = EnsureChannelFirst(channel_dim='no_channel')
metatensor_2d = multiChannel(metatensor_2d)
metatensor_3d = multiChannel(metatensor_3d)
print('Metatensors (sitk) after EnsureChannelFirst')
print(metatensor_2d.shape)
print(metatensor_3d.shape)

# Output images
sitk_output_2d = MetatensorToSitk(metatensor_2d)
show_image(sitk_output_2d, title= 'Image 2D (sitk origin)')
sitk_output_3d = MetatensorToSitk(metatensor_3d)
show_image(sitk_output_3d, title= 'Image 3D (sitk origin)')


# Now load from Nifti instead of using sitk object
loadNifti = LoadImage(image_only=True)
metatensor1_2d = loadNifti(img1_file_2d)
metatensor2_2d = loadNifti(img2_file_2d)
metatensor1_3d = loadNifti(img1_file_3d)
metatensor2_3d = loadNifti(img2_file_3d)
print('Metatensors from nifti before EnsureChannelFirst')
print(metatensor1_2d.shape)
print(metatensor1_3d.shape)

# Create dictionary for MONAI transforms
metatensor_2d = {'image_1': metatensor1_2d, 'image_2': metatensor2_2d}
metatensor_3d = {'image_1': metatensor1_3d, 'image_2': metatensor2_3d}
print(metatensor_2d)


# Pre-array
pre_array = [
        EnsureChannelFirstd(keys=['image_1', 'image_2']), 
        ConcatItemsd(keys=['image_1', 'image_2'], name='image')
    ]
pre_transform = Compose(pre_array)
metatensor_2d = pre_transform(metatensor_2d)
metatensor_3d = pre_transform(metatensor_3d)
print('Metatensors (nifti) after EnsureChannelFirst')
print(metatensor_2d['image_1'].shape)
print(metatensor_3d['image_1'].shape)

# Output images
sitk_output_2d = MetatensorToSitk(metatensor_2d['image'])
show_image(sitk_output_2d, title= 'Image 2D (nifti origin)')
sitk_output_3d = MetatensorToSitk(metatensor_3d['image'])
show_image(sitk_output_3d, title= 'Image 3D (nifti origin)')


Reading labeled images from: /Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…

SimpleITK object sizes
(192, 192, 1)
(192, 192, 3)
Metatensors for sitk before EnsureChannelFirst
torch.Size([192, 192, 1])
torch.Size([192, 192, 3])
Metatensors (sitk) after EnsureChannelFirst
torch.Size([1, 192, 192, 1])
torch.Size([1, 192, 192, 3])


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…

Metatensors from nifti before EnsureChannelFirst
torch.Size([192, 192, 1])
torch.Size([192, 192, 3])
{'image_1': metatensor([[[0.6790],
         [0.6640],
         [0.6646],
         ...,
         [0.0148],
         [0.0218],
         [0.0158]],

        [[0.6475],
         [0.6310],
         [0.6579],
         ...,
         [0.0117],
         [0.0185],
         [0.0099]],

        [[0.6291],
         [0.6100],
         [0.6505],
         ...,
         [0.0107],
         [0.0099],
         [0.0146]],

        ...,

        [[0.0132],
         [0.0117],
         [0.0224],
         ...,
         [0.0027],
         [0.0039],
         [0.0054]],

        [[0.0138],
         [0.0108],
         [0.0221],
         ...,
         [0.0031],
         [0.0028],
         [0.0070]],

        [[0.0169],
         [0.0094],
         [0.0180],
         ...,
         [0.0045],
         [0.0086],
         [0.0078]]]), 'image_2': metatensor([[[ 2.4066],
         [ 2.4380],
         [ 2.5111],
         ...,

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…